In [1]:
import pandas as pd

df = pd.read_excel("data.xlsx")



In [2]:
# Đọc dữ liệu từ file Excel và giữ lại các cột cần thiết
file_path = 'data_first.xlsx'  # Đường dẫn tới file Excel
df = pd.read_excel(file_path, usecols=['Bank', 'Link', 'Title', 'Description', 'Time_published', 'Evaluate( Human)'])

# Kiểm tra dữ liệu
print("Dữ liệu đầu vào:")
print(df.head())

Dữ liệu đầu vào:
     Bank                                               Link  \
0  ABBank  https://cafef.vn/abbank-giam-toi-15-nam-lai-su...   
1  ABBank  https://cafef.vn/abbank-va-maybank-tang-cuong-...   
2  ABBank  https://cafef.vn/16-ca-nhan-3-to-chuc-so-huu-g...   
3  ABBank  https://cafef.vn/abbank-dat-558-ty-dong-loi-nh...   
4  ABBank  https://cafef.vn/nhieu-uu-dai-danh-cho-khach-h...   

                                               Title  \
0  ABBank giảm tới 1,5%/năm lãi suất cho vay khác...   
1  Maybank tăng cường quan hệ hợp tác chiến lược,...   
2  16 cá nhân, 3 tổ chức sở hữu gần 67% vốn điều ...   
3  ABBANK đạt 558 tỷ đồng lợi nhuận trước thuế tr...   
4  Nhiều ưu đãi dành cho khách hàng ABBANK nhân s...   

                                         Description         Time_published  \
0  Với mong muốn cùng chung tay tiếp sức, hỗ trợ ...  25-09-2024 - 15:14 PM   
1  Vừa qua tại Hà Nội, Ngân hàng TMCP An Bình (AB...  11-09-2024 - 13:04 PM   
2  Ngân hàng TMCP An Bìn

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [6]:
# Hàm chuẩn hóa văn bản
def preprocess_text(text):
    if isinstance(text, str):  # Kiểm tra xem text có phải là chuỗi không
        text = text.lower()  # Chuyển về chữ thường
        text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ dấu câu
        return text
    return ""  # Trả về chuỗi rỗng nếu không phải là chuỗi

# Áp dụng chuẩn hóa cho Title và Description
df['Title'] = df['Title'].apply(preprocess_text)
df['Description'] = df['Description'].apply(preprocess_text)

# Kết hợp Title và Description
df['text'] = df['Title'] + " " + df['Description']

# Mã hóa nhãn
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Evaluate( Human)'])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Loại bỏ giá trị NaN nếu có
X_train = X_train.dropna()
y_train = y_train[X_train.index]
X_test = X_test.dropna()
y_test = y_test[X_test.index]

# Sử dụng TF-IDF để mã hóa
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Sử dụng SMOTE để cân bằng lớp
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_tfidf.toarray(), y_train)

# Tính toán trọng số cho các lớp
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_resampled),
    y=y_resampled
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Xây dựng mô hình DNN
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Số lớp tương ứng với số nhãn

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_resampled, y_resampled, epochs=10, batch_size=32, validation_split=0.2, class_weight=class_weights_dict)

# Đánh giá mô hình
loss, accuracy = model.evaluate(X_test_tfidf.toarray(), y_test)
print(f'Test Accuracy: {accuracy:.2f}')

c:\Users\FPT SHOP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6618 - loss: 0.9144 - val_accuracy: 0.2935 - val_loss: 1.4761
Epoch 2/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8557 - loss: 0.4167 - val_accuracy: 0.4560 - val_loss: 1.1665
Epoch 3/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8915 - loss: 0.3273 - val_accuracy: 0.5590 - val_loss: 1.0042
Epoch 4/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9111 - loss: 0.2725 - val_accuracy: 0.6233 - val_loss: 0.8466
Epoch 5/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9278 - loss: 0.2326 - val_accuracy: 0.6688 - val_loss: 0.7408
Epoch 6/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9366 - loss: 0.2031 - val_accuracy: 0.7247 - val_loss: 0.6469
Epoch 7/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9452 - loss: 0.1770 - val_accuracy: 0.7566 - val_loss: 0.5769
Epoch 8/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9531 - loss: 0.1555 - val_accuracy: 0.

In [9]:
# Đánh giá mô hình trên tập huấn luyện
train_loss, train_accuracy = model.evaluate(X_train_tfidf.toarray(), y_train)
print(f'Train Accuracy: {train_accuracy:.2f}')

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(X_test_tfidf.toarray(), y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')


456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9449 - loss: 0.1364
Train Accuracy: 0.95
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8242 - loss: 0.5964
Test Accuracy: 0.82


In [8]:
import numpy as np


def predict_title_and_description(title, description):
 
    title_processed = preprocess_text(title)
    description_processed = preprocess_text(description)
    text_input = title_processed + " " + description_processed
    
    # vector TF-IDF
    text_vector = vectorizer.transform([text_input]).toarray()  
    
    # Predict
    prediction = model.predict(text_vector)
    predicted_class = np.argmax(prediction, axis=-1) 
    
    return label_encoder.inverse_transform(predicted_class)[0]

new_title = "Tiền trong tài khoản “bốc hơi” vì điện thoại bị chiếm quyền: Ngân hàng chỉ cách bảo vệ an toàn"
new_description = "Để phòng ngừa rủi ro khách hàng bị chiếm đoạt tài khoản, mất tài sản, nhiều ngân hàng đã phát cảnh báo và chỉ ra cách bảo vệ tài khoản an toàn"
predicted_label = predict_title_and_description(new_title, new_description)

print(f'Dự đoán nhãn: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Dự đoán nhãn: Negative
